Homework #3


### General Notes about this assignment 

Data:
X_train.npy: training data numpy array. Each row in the array corresponds to an image unrolled to a vector (50 x 37 = 1850 dimension)

y_train.npy: labels (0-6) of each data corresponding to the image in the same row in X_train.npy

X_test.npy: testing data numpy array for evaluation of your models. The prediction outputs on this test data is to be submitted for scoreboard

Ideas:
- multiclass SVM (see Multiclass classification)
- kNN with PCA 
- CNN  
- Viola-Jones face detector

## Building and evaluating models

In [13]:
import numpy as np
import matplotlib.pyplot as pl
import pandas as pd 
%matplotlib inline
# np.set_printoptions(threshold=np.inf)

import sklearn
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
# from sklearn.decomposition import PCA
from scipy.stats import zscore
from scipy import stats
from time import time
# Size the plot appropriately for online display
pl.rcParams['figure.figsize'] = (12.0, 10.0)

In [2]:
""" PLEASE CHANGE THIS!!!! """
HIDDEN_LAYERS = (1851,)
# HIDDEN_LAYERS = (1851, 400,)
# HIDDEN_LAYERS = (1851, 400, 20,)

In [3]:
#Load dataset

#train data

xin= np.load("./X_train.npy")
yin= np.load("./y_train.npy")
xout= np.load("./X_test.npy")


In [4]:
# Helper functions
def createSubmission(filename, y):
    fo = open( filename , 'w' )
    fo.write("ImageId,PredictedClass\n")
    for i in range(y.shape[0]):
        fo.write(str(i)+","+str(y[i])+"\n")
    fo.close()

def nnPredict(model,x, y):
    ypredict= model.predict(x)
    score= f1_score(y , ypredict, average='macro');
    return score

#scale data
def normalize(x):
    r=zscore(x, axis=0)
    return r

In [5]:
# Rescale features to have mean 0 and variance 1
xin = normalize(xin)
xout = normalize(xout)

In [5]:
def buildModel(param_grid, x, y):
    t0 = time()
    print("Fitting the classifier to the training set")
    model = GridSearchCV(MLPClassifier(hidden_layer_sizes=HIDDEN_LAYERS,
                                       activation='logistic',
                                       solver='lbfgs',
                                       verbose=True,
                                       max_iter=1000*len(HIDDEN_LAYERS) + 2000,
                                       learning_rate_init=0.001
                                      ),
            param_grid, cv=StratifiedKFold(n_splits=3),
            n_jobs=-1, scoring='f1_macro')
    model = model.fit(x, y)
    
    print("done in %0.3fs" % (time() - t0))
    print("Best score found by grid search:")
    print(model.best_score_)
    print("Best params found by grid search:")
    print(model.best_params_)
    print("\n")
    import json
    with open('./buildModelResults.json', 'w') as f:
        f.write(str(model.cv_results_))
    return model

In [6]:
params={
        'alpha' : [5, 8, 10, 100]
        }
bestModel = buildModel(params, xin, yin)

Fitting the classifier to the training set
done in 28078.360s
Best score found by grid search:
0.810910991502
Best params found by grid search:
{'alpha': 5}




In [10]:
# Obtain the best parameters from the GridSearch
nn = MLPClassifier(hidden_layer_sizes=HIDDEN_LAYERS,
                   activation='logistic', 
                   solver='lbfgs', 
                   alpha=5,
                   verbose=True,
                   max_iter=2000
                  )

# Split the train and test
skf = StratifiedKFold(n_splits=2)
for train, test in skf.split(xin, yin):
    X_train = xin[train]
    y_train = yin[train]
    X_test  = xin[test]
    y_test  = yin[test]
    nn.fit(X_train, y_train)
    y_pred = nn.predict(X_test)
    print("Detailed classification report:")
    print()
    print(classification_report(y_test, y_pred))
    print()

Detailed classification report:

             precision    recall  f1-score   support

          0       0.71      0.58      0.64        26
          1       0.80      0.90      0.85        92
          2       0.82      0.74      0.78        43
          3       0.88      0.91      0.90       198
          4       0.77      0.70      0.73        43
          5       0.85      0.73      0.79        30
          6       0.85      0.83      0.84        53

avg / total       0.84      0.84      0.83       485


Detailed classification report:

             precision    recall  f1-score   support

          0       0.72      0.52      0.60        25
          1       0.82      0.82      0.82        91
          2       0.78      0.67      0.72        42
          3       0.83      0.90      0.86       197
          4       0.79      0.70      0.74        43
          5       0.86      0.83      0.85        30
          6       0.72      0.74      0.73        53

avg / total       0.80     

In [12]:
# Train the NN on all training data
# nn.fit(xin, yin)

"""PLEASE CHANGE THIS"""
NEURAL_NETWORK_ARCH = len(HIDDEN_LAYERS) # Number of hidden layers
DATE = '10_28'

# Run the trained neural network on the test set and write results to a csv file
filename = './submission_nn_' + DATE + '_' + str(NEURAL_NETWORK_ARCH) + '.csv'
y_out = nn.predict(xout)
createSubmission(filename, y_out)